# Eagle 1T - 2nd stage calibration 

Various smaller runs to figure out the optimal settings for the model calibration

In [2]:
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="Eagle-7B-2T-Calibration"
DEEPSPEED_STRAT="deepspeed_stage_2"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# The model sizing
MODEL_PATH="/workspace/main-models/RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/recursal/RWKV-infctx-trainer/notebook/trainer-v5-validation
TRAINER_DIR: /home/recursal/RWKV-infctx-trainer/RWKV-v5
PROJECT_DIR: /home/recursal/RWKV-infctx-trainer


# 1e-5 with NO CUDA + loss threshold

In [ ]:
!cd "{TRAINER_DIR}" && \
    export RWKV_NO_CUDA=1 && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/ref-3e-5.yaml" \
        --model.load_model="{MODEL_PATH}" \
        --model.lr_init=1e-5 \
        --model.lr_final=1e-5 \
        --model.token_loss_threshold=0.02 \
        --data.skip_datapath_setup=True \
        --trainer.callbacks.init_args.dirpath="/checkpoint/calibration/Eagle-2T-p1/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - 1e-5 : no-cuda+loss threshold ({DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.target_batch_size=1024 \
        --trainer.microbatch_size=8 \
        --model.ctx_len=4096 \
        --trainer.devices="{GPU_DEVICES}"